In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import math
import time
import xgboost as xgb
import gc
import pickle
from numpy import array

In [2]:
with open('train_XGB_1124', 'rb') as f:
    train_all = pickle.load(f)
with open('test_XGB_1124', 'rb') as f:
    test_all = pickle.load(f)

In [3]:
# display(train_all)

In [4]:
submission = pd.read_csv('files/test.csv')

In [5]:
submission.insert(0, 'predicted', 0)
submission.insert(0, 'meter_reading', 0)

In [4]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings

            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
#             print("dtype after: ",props[col].dtype)
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist


In [5]:
submission, NAlist = reduce_mem_usage(submission)

NameError: name 'submission' is not defined

In [ ]:
print(train_all.shape)
print(test_all.shape)
display(train_all)

In [ ]:
with pd.option_context('display.max_rows', None):
    print(test_all.dtypes)

In [ ]:
def evalerror(preds, actual):
    preds[preds<0] = 0
    return math.sqrt(np.square(np.log(preds+1)-np.log(actual+1)).mean())

In [ ]:
def learn_real(X, y, X_test, rounds):
#     rmsle to rmse
    y = np.log(y+1)
    params = {
        'objective' : 'reg:squarederror',
#         'nthread' : 8,
        'tree_method': 'gpu_hist',
        'eta' : 0.1,
        'min_child_weight' : 3,
#         'lambda' : 2
        'max_depth' : 8
        
    }
    num_rounds = rounds
    plst = params.items()
    dtrain = xgb.DMatrix( data = X.values, label = y.values)
    del X,y
    gc.collect()
    model = xgb.train(plst, dtrain, num_rounds)
    del dtrain
    gc.collect()
    # predict 
    l = X_test.shape[0]
    preds = np.array([])
    for i in range(5):
        a = int(l*i/5)
        b = int(l*(i+1)/5)
        X_test2 = X_test.iloc[a:b]
        dtest = xgb.DMatrix( data = X_test2.values)
        preds2 = model.predict(dtest)
        preds = np.concatenate((preds, preds2), axis=None)
#     rmsle to rmse
    preds = np.exp(preds) - 1
    preds[preds<0] = 0
    return preds


In [ ]:
def predict(train,test,rounds):
    tick = time.time()
    y_validall = []
    y_predsall = []

    y = train['meter_reading']
    train = train.drop('meter_reading', axis=1)
    #   for learn2
    
#     X_train, X_valid, y_train, y_valid = train_test_split(train, y, train_size=0.8, test_size=0.2, random_state=0)
#     (score,preds) = learn2(X_train, X_valid, y_train, y_valid )

#     y_validall.append(y_valid.values)
#     y_predsall.append(preds)
    
#     y_validall2 = np.concatenate( y_validall, axis=0 )
#     y_predsall2 = np.concatenate( y_predsall, axis=0 )
#     print(evalerror(y_validall2,y_predsall2))
    
#     tock = time.time()
#     print('Time used : {}'.format(tock-tick))
    
    #   for learn_real
    y_row = test['row_id']
    test = test.drop('row_id', axis=1)
    preds = learn_real(train, y, test, rounds)
    tock = time.time()
    print('Time used : {}'.format(tock-tick))
    
    return(y_row,preds)

    # print(y_predsall)
    # with open('y_predsall1105_2', 'wb') as f:
    #     pickle.dump(y_predsall, f)

In [ ]:
split_building = [778,1099]
split_siteid = [7]

In [6]:
drop_col = [
'building_id',
'meter',
'timestamp',
# 'meter_reading',
'site_id',
'cloud_coverage',
# 'precip_depth_1_hr',
# 'sea_level_pressure'
]

drop_col_valid = [
'building_id',
'meter',
# 'timestamp',
# 'meter_reading',
'site_id',
'cloud_coverage',
# 'precip_depth_1_hr',
# 'sea_level_pressure'
]

In [15]:
def learn_valid(X, rounds):
    tick = time.time()
    X1 = X.query('(timestamp <= "2016-05-01")')
    X2 = X.query('(timestamp >= "2016-09-01")')
    y1 = X1['meter_reading']
    y2 = X2['meter_reading']
    X1 = X1.drop(['meter_reading','timestamp'],axis=1)
    X2 = X2.drop(['meter_reading','timestamp'],axis=1)
    y1 = np.log(y1+1)
    y2 = np.log(y2+1)
    params = {
        'objective' : 'reg:squarederror',
#         'nthread' : 8,
        'tree_method': 'gpu_hist',
        'eta' : 0.1,
        'min_child_weight' : 3,
#         'lambda' : 2
        'max_depth' : 8
        
    }
    plst = params.items()
    dtrain = xgb.DMatrix( data = X1.values, label = y1.values)
    dtest = xgb.DMatrix( data = X2.values)
#     num_rounds = 100
    model = xgb.train(plst, dtrain, rounds)

    # predict 
    y2_preds = model.predict(dtest)
    y2_preds = np.exp(y2_preds) - 1
    y2 = np.exp(y2) - 1
    y2_preds[y2_preds<0] = 0
    score = evalerror(y2,y2_preds)
    print(score)
    tock = time.time()
    print(tock-tick)
    return (y2,y2_preds)


In [16]:
gc.collect()

20

In [17]:
y2_all = np.array([])
y2_allpreds =  np.array([])
for meter in range(0,4):
    print('Meter {}'.format(meter))
    train_meter = train_all[train_all['meter'] == meter]
    train = train_meter[~train_meter['building_id'].isin(split_building)]
    train = train[~train['site_id'].isin(split_siteid)]
    
    test_meter = test_all[test_all['meter'] == meter]
    test = test_meter[~test_meter['building_id'].isin(split_building)]
    test = test[~test['site_id'].isin(split_siteid)]
    del train_meter
    del test_meter
    gc.collect()
#     for validation
#     train.drop(drop_col_valid,axis=1,inplace=True)
#     (y2,y2_preds) = learn_valid(train,500)
#     y2_all = np.concatenate((y2_all, y2), axis=None) 
#     y2_allpreds = np.concatenate((y2_allpreds, y2_preds), axis=None)
#     for learn
    train.drop(drop_col,axis=1,inplace=True)
    test.drop(drop_col,axis=1,inplace=True)
    (row,preds) = predict(train,test,500)
    del train,test
    gc.collect()

    row = row.values.astype(int)
    submission.at[row,'predicted'] = 1
    submission.at[row,'meter_reading'] = preds
    del row, preds
    gc.collect()

Meter 0
Time used : 211.14210605621338
Meter 1
Time used : 81.20705080032349
Meter 2
Time used : 58.287596225738525
Meter 3
Time used : 35.40911936759949


In [18]:
# for split_id
for building in split_building:
    for meter in range(0,4):
        train = train_all[train_all['building_id'] == building]
        test = test_all[test_all['building_id'] == building]
        train = train[train['meter'] == meter]
        test = test[test['meter'] == meter]
        if train.shape[0] == 0:
            continue
        print(building,meter)
        print(train.shape[0])
        train.drop(drop_col,axis=1,inplace=True)
        test.drop(drop_col,axis=1,inplace=True)
#         remove col with one value
        nunique = train.apply(pd.Series.nunique)
        cols_to_drop = nunique[nunique == 1].index
        train.drop(cols_to_drop, axis=1)
        test.drop(cols_to_drop, axis=1)
#         
        (row, preds) = predict(train, test,500)
        del train, test
        gc.collect()
        row = row.values.astype(int)
        submission.at[row,'predicted'] = 1
        submission.at[row,'meter_reading'] = preds
        del row, preds
        gc.collect()

778 0
8773
Time used : 6.453373670578003
778 1
8084
Time used : 6.646116018295288
1099 0
8781
Time used : 6.677650690078735
1099 2
8783
Time used : 6.505289316177368


In [19]:
# for split_siteid
for site_id in split_siteid:
    for meter in range(0,4):
        train = train_all[train_all['site_id'] == site_id]
        test = test_all[test_all['site_id'] == site_id]
        train = train[train['meter'] == meter]
        test = test[test['meter'] == meter]
        if train.shape[0] == 0:
            continue
        print(meter,train.shape[0])
        train.drop(drop_col,axis=1,inplace=True)
        test.drop(drop_col,axis=1,inplace=True)
#         remove col with one value
        nunique = train.apply(pd.Series.nunique)
        cols_to_drop = nunique[nunique == 1].index
        train.drop(cols_to_drop, axis=1)
        test.drop(cols_to_drop, axis=1)
#         
        (row, preds) = predict(train, test,500)
        del train, test
        gc.collect()
        row = row.values.astype(int)
        submission.at[row,'predicted'] = 1
        submission.at[row,'meter_reading'] = preds
        del row, preds
        gc.collect()

0 102745
Time used : 10.397579908370972
1 128456
Time used : 11.432095766067505
2 102607
Time used : 10.565889120101929
3 25834
Time used : 7.5759758949279785


In [20]:
with open('sub', 'wb') as f:
    pickle.dump(submission, f)